In [ ]:
import os

if "src" not in os.listdir():
    os.chdir("../../../")

from datetime import timedelta
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import keras.backend as K
from keras import Model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.losses import MeanSquaredError
from keras.optimizers import Adam

%load_ext autoreload
%autoreload 2

from src.lib.stock_dataset import read_stock_dataset
from src.lib.activations import negative_softmax
from src.lib.losses import negative_profit_loss, multi_negative_sharpe_ratio_loss

In [ ]:
symbols = ["SPY", "SPXS"]
n_time_steps = 64
batch_size = 200
epochs = 3
learning_rate = 0.001

In [ ]:
dataset = read_stock_dataset(symbols, "oc_ret", n_time_steps)
train, valid, test = dataset.train_valid_test_split(timedelta(days=365), timedelta(days=365), scaled=True)
n_features = train.n_features
n_symbols = train.n_symbols
print(f"Loaded {len(train)} training, {len(valid)}, validation, and {len(test)} testing samples")
print(f"{n_features} features")
print(f"{n_symbols} targets")

In [ ]:
def build_model():
    target_tensor = Input((n_symbols,))

    # LSTM Model
    mono_input_tensor = Input((n_time_steps, n_features))
    z = mono_input_tensor
    lstm_out = LSTM(64)(z)

    # Holdings head
    z = Dropout(0.25)(lstm_out)
    z = Dense(64, activation="selu")(z)
    z = Dropout(0.1)(z)
    holdings = Dense(n_symbols, activation=negative_softmax, name="holdings")(z)
    
    # Percent Allocation head
    z = Dropout(0.25)(lstm_out)
    z = Dense(64, activation="selu")(z)
    z = Dropout(0.1)(z)
    p_invest = Dense(1, activation="sigmoid", name="p_invest")(z)

    # Combine both heads
    adj_holdings = holdings * p_invest

    # That's the model
    mono_model = Model([mono_input_tensor, target_tensor], adj_holdings)

    # Apply the sharpe loss to the holdings head
    sharpe_loss = multi_negative_sharpe_ratio_loss(target_tensor, holdings)
    mono_model.add_loss(sharpe_loss)

    # Apply the profit loss to the allocation head
    neg_adj_profit = K.sum(negative_profit_loss(target_tensor, adj_holdings), axis=-1)
    mono_model.add_loss(neg_adj_profit)
    # mono_model.compile(Adam(lr=learning_rate))
    
    # A model that runs the other model over two samples next to each other in time
    bi_feature_tensor = Input((2, n_time_steps, n_features), name="bi_features")
    bi_target_tensor = Input((2, n_symbols), name="bi_targets")

    X_initial = bi_feature_tensor[:, 0, :, :]
    y_initial = bi_target_tensor[:, 0, :]

    X_final = bi_feature_tensor[:, 1, :, :]
    y_initial = bi_target_tensor[:, 1, :]

    holdings_initial = K.stop_gradient(mono_model([X_initial, y_initial]))
    holdings_final = mono_model([X_final, y_initial])

    # Calculate the squared difference between the two outputs
    holding_diff =  0.0001 * K.mean(K.square(holdings_final - holdings_initial), axis=-1)

    bi_model = Model([bi_feature_tensor, bi_target_tensor], holding_diff)

    bi_model.add_loss(holding_diff)

    bi_model.compile(Adam(lr=learning_rate))

    # bi_model.summary()

    # One more model that simply outputs the desired holdings given an input
    pred_model = Model(mono_input_tensor, adj_holdings)

    return bi_model, pred_model

In [ ]:
train_batchable_index = train.get_batchable_index(trade_market_open=True)
n_iter_per_epoch = len(train_batchable_index)//batch_size

loss_history = []

model, pred_model = build_model()
valid_batchable_index = valid.get_batchable_index(trade_market_open=True)
# X_batch_valid, y_batch_valid = valid.get_paired_batch(len(valid_batchable_index)-1, valid_batchable_index, shuffle=False)

for epoch in range(epochs):
    pbar = tqdm(range(n_iter_per_epoch))
    for i in pbar:

        X_batch, y_batch = train.get_paired_batch(batch_size, train_batchable_index, shuffle=True)
        loss = model.train_on_batch([X_batch, y_batch])
        loss_history.append(loss)

        if len(loss_history) > 150:
            avg_loss = np.mean(loss_history[-150:])
        else:
            avg_loss = np.mean(loss_history)
        
        pbar.set_description("loss=%.3f" % avg_loss)
    
    # Evaluating the model kills the kernel
    # losses_valid = model.evaluate([X_batch_valid, y_batch_valid])
    # avg_loss_valid = np.mean(losses_valid)
    # print("Valid Loss:", -avg_loss_valid)

In [ ]:
X_batch_valid_pred, y_batch_valid_pred = valid.get_batch(len(valid_batchable_index), valid_batchable_index, shuffle=False)

preds = pred_model.predict(X_batch_valid_pred)
for i, ticker in enumerate(symbols):
    print(ticker)
    plt.hist(preds[:, i], bins=100)
    plt.show()

print("Allocation")
plt.hist(np.abs(preds).sum(-1), bins=100)
plt.show()

In [ ]:
x = preds[:, 0]
y = preds[:, 1]

heatmap, xedges, yedges = np.histogram2d(x, y, bins=256)
# heatmap = np.log(heatmap+1)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
fig = go.Figure()
fig.add_trace(go.Heatmap(z=heatmap.T))
fig.show()

In [ ]:
rets = np.exp((np.log(preds * y_batch_valid_pred+1)).sum(axis=-1).cumsum())
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid.index, y=rets))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid.index, y=preds[:, 0], name=symbols[0]))
fig.add_trace(go.Scatter(x=valid.index, y=preds[:, 1], name=symbols[1]))
fig.show()